In [40]:
import yaml
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
# import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch.autograd import Variable

In [17]:
def get_keyframes(downsampled_video_filename, keyframe_interval): 
    "Generator function which returns the next keyframe."         

    # Create video capture object                                     
    video_cap = cv2.VideoCapture(downsampled_video_filename)                                     
    frame = 0                                                                                    
    while True:                                                                                  
        frame += 1                                                                               
        ret, img = video_cap.read()                                                              
        if ret is False:                                                                         
            break                                                                                
        if frame % keyframe_interval == 0:
              yield img
    video_cap.release()

In [18]:
all_video_names = "tryvideos"
config_file = "config.yaml"                                                                  
my_params = yaml.load(open(config_file))                                                   
                                                                                           
# Get parameters from config file                                                          
keyframe_interval = my_params.get('keyframe_interval')                                                                          
cnn_features_folderpath = my_params.get('cnn_features')                                  
downsampled_videos = my_params.get('downsampled_videos')

In [19]:
# fread = open(all_video_names, "r")
# for line in fread.readlines():
line = "HVC1012"

start = time.time()
video_name = line.replace('\n', '')
downsampled_video_filename = os.path.join(downsampled_videos, video_name + '.ds.mp4')
cnn_feat_video_filename = os.path.join(cnn_features_folderpath, video_name + '.cnnn')

if (os.path.isfile(cnn_feat_video_filename)) or (not os.path.isfile(downsampled_video_filename)):
    print("skipping", video_name)
#     continue
start = time.time()
keyframe_iterator = get_keyframes(downsampled_video_filename, keyframe_interval)
print("iterator: ", time.time() - start)

iterator:  0.0002124309539794922


In [20]:
resnet_model = models.resnet18(pretrained=True)

layer = resnet_model._modules.get('avgpool')

In [21]:
resnet_model.eval()
model = resnet_model

In [22]:
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [23]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.fromarray(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Defining a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.squeeze())
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    resnet_model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding

In [38]:
# Image.fromarray(img).show()

feat = np.vstack([get_vector(img) for img in keyframe_iterator])

In [39]:
feat.shape

(37, 512)

In [35]:
# # Unused Code

# model = models.alexnet(pretrained=True)

# # remove last fully-connected layer
# # new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
# # model.classifier = new_classifier


# original_model = model

# class AlexNetConv4(nn.Module):
#             def __init__(self):
#                 super(AlexNetConv4, self).__init__()
#                 self.features = nn.Sequential(
#                     # stop at conv4
#                     *list(original_model.features.children())[:-1]
#                 )
#             def forward(self, x):
#                 x = self.features(x)
#                 return x

# model = AlexNetConv4()

# # Data augmentation and normalization for training
# # Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

# mytransform = transforms.Compose([
# transforms.Resize(256),
# transforms.CenterCrop(224),
# transforms.ToTensor(),
# transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [32]:

# model.eval()
# # pil_image = Image.fromstring("L", cv2.GetSize(img), img.tostring())
# pil_image = Image.fromarray(img)
# transformed_image = mytransform(pil_image).unsqueeze_(0)


# # model(torch.Tensor(mytransform(img)))

# feat = model(Variable(transformed_image))